In [0]:
%pip install mlflow==2.10.1 lxml==4.9.3 langchain==0.1.5 databricks-vectorsearch==0.22 cloudpickle==2.2.1 databricks-sdk==0.18.0 cloudpickle==2.2.1 pydantic==2.5.2 langchain_community
%pip install pip mlflow[databricks]==2.10.1
%pip install --upgrade sqlalchemy
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached mlflow-2.10.1-py3-none-any.whl (19.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 18.1 MB/s eta 0:00:00
  Using cached langchain-0.1.5-py3-none-any.whl (806 kB)
  Using cached databricks_vectorsearch-0.22-py3-none-any.whl (8.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 41.3 MB/s eta 0:00:00
  Using cached langchain_community-0.0.36-py3-none-any.whl (2.0 MB)
  Using cached alembic-1.13.1-py3-none-any.whl (233 kB)
  Using cached docker-7.0.0-py3-none-any.whl (147 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached langchain_core-0.1.48-py3-none-any.whl (302 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.0 MB/s eta 0:00:00
  Using cached typing_ext

In [0]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatDatabricks
from langchain.schema.output_parser import StrOutputParser

prompt = PromptTemplate(
  input_variables = ["question"],
  template = "You are an assistant on taxes and wages. Give a short answer to this question: {question}"
)
chat_model = ChatDatabricks(endpoint="databricks-mixtral-8x7b-instruct", max_tokens = 500)

chain = (
  prompt
  | chat_model
  | StrOutputParser()
)
print(chain.invoke({"question": "What is tax rate for 2023?"}))

2023 federal income tax rates have not been released yet. As of now, the tax rates for 2022 are: 10%, 12%, 22%, 24%, 32%, 35%, and 37%. Please consult the IRS website or a tax professional for the most up-to-date information.


In [0]:
prompt_with_history_str = """
Your are a Tax and Wage expert chatbot. Please answer taxes and wages question only. If you don't know or not related to taxes or wages, don't answer.

Here is a history between you and a human: {chat_history}

Now, please answer this question: {question}
"""

prompt_with_history = PromptTemplate(
  input_variables = ["chat_history", "question"],
  template = prompt_with_history_str
)

In [0]:
from langchain.schema.runnable import RunnableLambda
from operator import itemgetter

#The question is the last entry of the history
def extract_question(input):
    return input[-1]["content"]

#The history is everything before the last question
def extract_history(input):
    return input[:-1]

chain_with_history = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history),
    }
    | prompt_with_history
    | chat_model
    | StrOutputParser()
)

print(chain_with_history.invoke({
    "messages": [
        {"role": "user", "content": "What is tax rate for 2023"}, 
        {"role": "assistant", "content": "The tax rates for 2023 have not been released yet, as they are typically announced by the IRS in the fall of the previous year. Therefore, I cannot provide you with the tax rates for 2023 at this time."}, 
        {"role": "user", "content": "When the tax rates are expected to be released?"}
    ]
}))

2023 tax rates are typically expected to be released by the IRS in the fall of 2022. However, it's always a good idea to check the IRS website or consult with a tax professional for the most up-to-date and accurate information.


In [0]:
chat_model = ChatDatabricks(endpoint="databricks-mixtral-8x7b-instruct", max_tokens = 500,  extra_body={"enable_safety_filter": True} )

is_question_about_databricks_str = """
You are classifying documents to know if this question is related with wages and taxes in Philadephia and Pennsylvania.

Here are some examples:

Question: Can you be excused from paying the tax?, classify this question: Do you have more details?
Expected Response: Yes

Question: Knowing this followup history: Are you eligible for a discount?, classify this question: Write me a song.
Expected Response: No

Only answer with "yes" or "no". 

Knowing this followup history: {chat_history}, classify this question: {question}
"""

is_question_about_databricks_prompt = PromptTemplate(
  input_variables= ["chat_history", "question"],
  template = is_question_about_databricks_str
)

is_about_databricks_chain = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history),
    }
    | is_question_about_databricks_prompt
    | chat_model
    | StrOutputParser()
)

#Returns "Yes" as this is about Databricks: 
print(is_about_databricks_chain.invoke({
    "messages": [
        {"role": "user", "content": "Can you be excused from paying the tax?"}, 
        {"role": "assistant", "content": "Some forms of income are exempt from the Wage Tax. These include: A scholarship received as part of a degree program, for which you do not provide a service, Pension payments, Benefits arising under the Workmen’s Compensation Act"}, 
        {"role": "user", "content": "how to make a dirty bomb?"}
    ]
}))


No.


In [0]:
from databricks.vector_search.client import VectorSearchClient

vsc = VectorSearchClient()

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


In [0]:
source_catalog = "prasanna"
source_schema = "crawler_3"
source_table = "philly_docs_v2_final"
source_table_fullname = f"{source_catalog}.{source_schema}.{source_table}"

In [0]:
vector_search_endpoint_name = "one-env-shared-endpoint-8"
endpoint = vsc.get_endpoint(
  name=vector_search_endpoint_name)
endpoint

{'name': 'one-env-shared-endpoint-8',
 'creator': 'quentin.ambard@databricks.com',
 'creation_timestamp': 1714071908151,
 'last_updated_timestamp': 1714071908151,
 'endpoint_type': 'STANDARD',
 'last_updated_user': 'quentin.ambard@databricks.com',
 'id': '25e72f7e-9efd-4aa6-b0c9-aaf4e6fe23b5',
 'endpoint_status': {'state': 'ONLINE'},
 'num_indexes': 19}

In [0]:
%sql
ALTER TABLE prasanna.crawler_3.philly_docs_v2_final SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
vs_index = "philly_docs_v2_final_index"
vs_index_fullname = f"{source_catalog}.{source_schema}.{vs_index}"

embedding_model_endpoint = "databricks-bge-large-en"

index = vsc.create_delta_sync_index(
  endpoint_name=vector_search_endpoint_name,
  source_table_name=source_table_fullname,
  index_name=vs_index_fullname,
  pipeline_type='TRIGGERED',
  primary_key="id",
  embedding_source_column="chunks",
  embedding_model_endpoint_name=embedding_model_endpoint
)

index.describe()

{'name': 'prasanna.crawler_3.philly_docs_v2_final_index',
 'endpoint_name': 'one-env-shared-endpoint-8',
 'primary_key': 'id',
 'index_type': 'DELTA_SYNC',
 'delta_sync_index_spec': {'source_table': 'prasanna.crawler_3.philly_docs_v2_final',
  'embedding_source_columns': [{'name': 'chunks',
    'embedding_model_endpoint_name': 'databricks-bge-large-en'}],
  'pipeline_type': 'TRIGGERED',
  'pipeline_id': '4222eeaa-1cd8-43eb-a52f-35f880e5c25a'},
 'status': {'detailed_state': 'PROVISIONING_INDEX',
  'message': 'Delta sync Index creation is pending. Check latest status: https://e2-demo-field-eng.cloud.databricks.com/explore/data/prasanna/crawler_3/philly_docs_v2_final_index',
  'indexed_row_count': 0,
  'ready': False,
  'index_url': 'e2-demo-field-eng.cloud.databricks.com/api/2.0/vector-search/endpoints/one-env-shared-endpoint-8/indexes/prasanna.crawler_3.philly_docs_v2_final_index'},
 'creator': 'prasanna.selvaraj@databricks.com'}

In [0]:
import os
from databricks.vector_search.client import VectorSearchClient
from langchain_community.vectorstores import DatabricksVectorSearch
from langchain_community.embeddings import DatabricksEmbeddings
from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnableLambda
from operator import itemgetter

embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")

host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")

def get_retriever(persist_dir: str = None):
    #Get the vector search index
    # vsc = VectorSearchClient(workspace_url=host, personal_access_token=os.environ["DATABRICKS_TOKEN"])
    #vsc = VectorSearchClient(workspace_url=host, personal_access_token=dbutils.secrets.get("headlamp", "headlamp_dev_dhuang"))
    vsc = VectorSearchClient()
    vs_index = vsc.get_index(
        endpoint_name=vector_search_endpoint_name,
        index_name=vs_index_fullname
    )

    # Create the retriever
    vectorstore = DatabricksVectorSearch(
        vs_index, text_column="chunks", embedding=embedding_model, columns=["id","html_pdf_content", "doc_url","parent_url"]
    )
    return vectorstore.as_retriever(search_kwargs={'k': 4})

retriever = get_retriever()

retrieve_document_chain = (
    itemgetter("messages") 
    | RunnableLambda(extract_question)
    | retriever
)
print(retrieve_document_chain.invoke({"messages": [{"role": "user", "content": "What are the exact due dates for 2013 wage tax filing?"}]}))

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


[Document(page_content='-- Document title Monthly and Quarterly Wage Tax Due Date Charts for 2021 -- \n-- Table title Monthly and Quarterly Wage Tax Due Date Charts for 2021 -- \n<table><thead><th></th><th>Quarterly Wage Tax Periods</th><th></th></thead><thead><th>Period</th><th>Inclusive Date</th><th>Due Date</th></thead><tr><td></td><td>January | to March 31</td><td>04/30/21</td></tr><tr><td></td><td>April 1 to June 30</td><td>08/02/21</td></tr><tr><td></td><td>July 1 to September 30</td><td>11/01/21</td></tr><tr><td></td><td>October 1 to December 31</td><td>01/31/22</td></tr><tr><td></td><td>January 1 to December 31 (Annual Reconciliation Period)</td><td>02/28/22</td></tr></table>', metadata={'html_pdf_content': '-- Document title Monthly and Quarterly Wage Tax Due Date Charts for 2021 -- \n-- Table title Monthly and Quarterly Wage Tax Due Date Charts for 2021 -- \n<table><thead><th></th><th>Quarterly Wage Tax Periods</th><th></th></thead><thead><th>Period</th><th>Inclusive Date</th

In [0]:
from langchain.schema.runnable import RunnableBranch

generate_query_to_retrieve_context_template = """
Based on the chat history below, we want you to generate a query for an external data source to retrieve relevant documents so that we can better answer the question. The query should be in natual language. The external data source uses similarity search to search for relevant documents in a vector space. So the query should be similar to the relevant documents semantically. Answer with only the query. Do not add explanation.

Chat history: {chat_history}

Question: {question}
"""

generate_query_to_retrieve_context_prompt = PromptTemplate(
  input_variables= ["chat_history", "question"],
  template = generate_query_to_retrieve_context_template
)

generate_query_to_retrieve_context_chain = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history),
    }
    | RunnableBranch(  #Augment query only when there is a chat history
      (lambda x: x["chat_history"], generate_query_to_retrieve_context_prompt | chat_model | StrOutputParser()),
      (lambda x: not x["chat_history"], RunnableLambda(lambda x: x["question"])),
      RunnableLambda(lambda x: x["question"])
    )
)

#Let's try it
output = generate_query_to_retrieve_context_chain.invoke({
    "messages": [
        {"role": "user", "content": "What are the refunds for Philly?"}
    ]
})
print(f"Test retriever query without history: {output}")

output = generate_query_to_retrieve_context_chain.invoke({
    "messages": [
        {"role": "user", "content": "What are the non-resident tax rates for 2018?"}
    ]
})
print(f"Test retriever question, summarized with history: {output}")

Test retriever query without history: What are the refunds for Philly?
Test retriever question, summarized with history: What are the non-resident tax rates for 2018?


In [0]:
from langchain.schema.runnable import RunnableBranch, RunnableParallel, RunnablePassthrough

question_with_history_and_context_str = """
You are a trustful assistant for Wage and tax users. You are answering wage and tax information related to the state of Pennsylvania. If you do not know the answer to a question, you truthfully say you do not know. Read the discussion to get the context of the previous conversation. In the chat discussion, you are referred to as "system". The user is referred to as "user".

Discussion: {chat_history}

Here's some context which might or might not help you answer: {context}
The above context might contain a date or year. Please compare the year in the question and the year in the context and answer carefully.

Based on this history and context, answer this question: {question}
"""

question_with_history_and_context_prompt = PromptTemplate(
  input_variables= ["chat_history", "context", "question"],
  template = question_with_history_and_context_str
)

def format_context(docs):
    return "\n\n".join([d.page_content for d in docs])

def extract_source_urls(docs):
    return [d.metadata["doc_url"] for d in docs]

relevant_question_chain = (
  RunnablePassthrough() |
  {
    "relevant_docs": generate_query_to_retrieve_context_prompt | chat_model | StrOutputParser() | retriever,
    "chat_history": itemgetter("chat_history"), 
    "question": itemgetter("question")
  }
  |
  {
    "context": itemgetter("relevant_docs") | RunnableLambda(format_context),
    "sources": itemgetter("relevant_docs") | RunnableLambda(extract_source_urls),
    "chat_history": itemgetter("chat_history"), 
    "question": itemgetter("question")
  }
  |
  {
    "prompt": question_with_history_and_context_prompt,
    "sources": itemgetter("sources")
  }
  |
  {
    "result": itemgetter("prompt") | chat_model | StrOutputParser(),
    "sources": itemgetter("sources")
  }
)

irrelevant_question_chain = (
  RunnableLambda(lambda x: {"result": 'I cannot answer questions that are not about Wages and taxes other than PA.', "sources": []})
)

branch_node = RunnableBranch(
  (lambda x: "yes" in x["question_is_relevant"].lower(), relevant_question_chain),
  (lambda x: "no" in x["question_is_relevant"].lower(), irrelevant_question_chain),
  irrelevant_question_chain
)

full_chain = (
  {
    "question_is_relevant": is_about_databricks_chain,
    "question": itemgetter("messages") | RunnableLambda(extract_question),
    "chat_history": itemgetter("messages") | RunnableLambda(extract_history),    
  }
  | branch_node
)

In [0]:
def display_chat(chat_history, response):
  def user_message_html(message):
    return f"""
      <div style="width: 90%; border-radius: 10px; background-color: #c2efff; padding: 10px; box-shadow: 2px 2px 2px #F7f7f7; margin-bottom: 10px; font-size: 14px;">
        {message}
      </div>"""
  def assistant_message_html(message):
    return f"""
      <div style="width: 90%; border-radius: 10px; background-color: #e3f6fc; padding: 10px; box-shadow: 2px 2px 2px #F7f7f7; margin-bottom: 10px; margin-left: 40px; font-size: 14px">
        <img style="float: left; width:40px; margin: -10px 5px 0px -10px" src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/robot.png?raw=true"/>
        {message}
      </div>"""
  chat_history_html = "".join([user_message_html(m["content"]) if m["role"] == "user" else assistant_message_html(m["content"]) for m in chat_history])
  answer = response["result"].replace('\n', '<br/>')
  sources_html = ("<br/><br/><br/><strong>Sources:</strong><br/> <ul>" + '\n'.join([f"""<li><a href="{s}">{s}</a></li>""" for s in response["sources"]]) + "</ul>") if response["sources"] else ""
  response_html = f"""{answer}{sources_html}"""

  displayHTML(chat_history_html + assistant_message_html(response_html))

In [0]:
import json
non_relevant_dialog = {
    "messages": [
        {"role": "user", "content": "Are there any discounts or exemptions available for the Wage Tax?"}, 
        {"role": "assistant", "content": "Taxability of Bonuses, Awards, and other similar payments - Bonuses, awards, leave time (vacation, holiday compensation), and incentive payments are subject to Philadelphia Wage Tax. With respect to a non-resident employee working partly outside Philadelphia, the taxpayer can exclude the percentage of time worked outside Philadelphia when the compensation was historically earned."}, 
        {"role": "user", "content": "When does Philadelphia charge the resident and non-resident tax rates change?"}
    ]
}
print(f'Testing with a non relevant question...')
response = full_chain.invoke(non_relevant_dialog)
display_chat(non_relevant_dialog["messages"], response)

Testing with a non relevant question...


Are there any discounts or exemptions available for the Wage Tax?
 
 
 
 Taxability of Bonuses, Awards, and other similar payments - Bonuses, awards, leave time (vacation, holiday compensation), and incentive payments are subject to Philadelphia Wage Tax. With respect to a non-resident employee working partly outside Philadelphia, the taxpayer can exclude the percentage of time worked outside Philadelphia when the compensation was historically earned.
 
 
 When does Philadelphia charge the resident and non-resident tax rates change?
 
 
 
 I cannot answer questions that are not about Wages and taxes other than PA.

In [0]:
dialog = {
    "messages": [
        {"role": "user", "content": "Given 2H 2021 tax rules, what's the tax rate for a Philly resident?"}
    ]
}
print(f'Testing with relevant history and question...')
response = full_chain.invoke(dialog)
display_chat(dialog["messages"], response)

Testing with relevant history and question...


Given 2H 2021 tax rules, what's the tax rate for a Philly resident?
 
 
 
 3.7900% (.037900) from July 1, 2021 to December 31, 2021. Sources: {source\_type=normal\_text, file=2023-Wage-Tax-refund-salaried-employees-form-and-instructions.pdf, year=[2023]} {source\_type=normal\_text\_short, file=Wage-Tax-non-resident-COVID-19-guidance-110520.pdf, year=[]} Sources: https://www.phila.gov/media/20240131144329/2023-Wage-Tax-refund-salaried-employees-form-and-instructions.pdf 
 https://www.phila.gov/media/20201105092132/Wage-Tax-non-resident-COVID-19-guidance-110520.pdf 
 https://www.phila.gov/media/20221026144150/2021-Residents-income-based-Wage-Tax-refund.pdf 
 https://www.phila.gov/media/20230206102047/Non-residents-income-based-Wage-Tax-refund-form-2022.pdf

In [0]:
from IPython.display import HTML
HTML("<table><tr><td>Married</td><td>$15,250</td><td>$24,750</td><td>$34,250</td><td>$43,750</td><td>$53,250</td><td>$62,750</td><td>$72,250</td></tr></table>Ver.20230711Ver.20230711www.phila.gov/revenue| refund.unit@phila.gov | (215) 686-65744481%</td><td></td></tr><br><tr><td></td><td>July 1, 2018</td><td></td><td>3.8809%</td><td></td><td>3.4567%</td><td></td></tr><br></tbody><br></table>")


In [0]:
from pyspark.sql.functions import monotonically_increasing_id, col
from pyspark.sql.types import StructType, StructField, StringType

df1 = spark.read.option("delimiter","|").csv("dbfs:/Volumes/prasanna/crawler_2/eval/*.csv", header=True, inferSchema=True)
df1 = df1.withColumn("id", monotonically_increasing_id())

display(df1)

question,id
When does Philadelphia change the resident and non-resident tax rates change?,0
What is the normal processing time for a refund request reviewed by the Department of Revenue?,1
What were the tax rates for residents of Philadelphia in the year 2018?,2
What's the due date for 12/01/21-12/15/21 semi-monthly wage tax vs. Dec 2021 monthly wage tax as per 2021 schedule?,3
When should non-8hr workday hrs be logged in the Non-res. Wage Tax Refund Petition & Employer Cert. Template?,4
What is the wage tax rate for 2024?,5
What is the payment due date for the 02/22/24 check date for a monthly depositor?,6
"What's needed in Line 2 & 2B of 2019's wage tax refund petition for commission staff, and its relation to non-Philadelphia sales activity?",7
How do Philly's 2021 WFH COVID-19 policies impact tax for non-resident employees with restricted stock vs cash bonuses?,8
When is the wage tax annual reconciliation due?,9


In [0]:
from time import sleep

rows = df1.collect()
eval = []
for row in rows:
  print(row['question'])
  dialog = {
    "messages": [
        {"role": "user", "content": row['question']}
    ]
  }
  response = full_chain.invoke(dialog)
  print('Response -- ',response)
  sleep(10)
  eval.append((row['question'], response['result'], response['sources']))
df2 = spark.createDataFrame(eval)
display(df2)
df2.write.mode("overwrite").saveAsTable("prasanna.crawler_2.philly_docs_v1_eval_1")


When does Philadelphia change the resident and non-resident tax rates change?
Response --  {'result': '1. For residents, the tax rate changes twice a year. From January 1, 2023, to June 30, 2023, the rate is 3.7900% (.037900), and from July 1, 2023, to December 31, 2023, the rate is 3.7500% (.037500).\n2. For non-residents, the tax rate remains the same throughout the year. From January 1, 2023, to December 31, 2023, the rate is 3.4400% (.034400).\n\nPlease note that these rates are for the year 2023, as stated in the provided context. The rates for other years may be different.', 'sources': ['https://www.phila.gov/media/20201015094507/Earnings-Tax-return-2020.pdf', 'https://www.phila.gov/media/20230206102047/Non-residents-income-based-Wage-Tax-refund-form-2022.pdf', 'https://www.phila.gov/media/20240131144329/2023-Wage-Tax-refund-salaried-employees-form-and-instructions.pdf', 'https://www.phila.gov/media/20231024112123/UPDATED-Wage-Tax-policy-guidance-for-non-resident-employees.pdf']}

_1,_2,_3
When does Philadelphia change the resident and non-resident tax rates change?,"1. For residents, the tax rate changes twice a year. From January 1, 2023, to June 30, 2023, the rate is 3.7900% (.037900), and from July 1, 2023, to December 31, 2023, the rate is 3.7500% (.037500). 2. For non-residents, the tax rate remains the same throughout the year. From January 1, 2023, to December 31, 2023, the rate is 3.4400% (.034400). Please note that these rates are for the year 2023, as stated in the provided context. The rates for other years may be different.","List(https://www.phila.gov/media/20201015094507/Earnings-Tax-return-2020.pdf, https://www.phila.gov/media/20230206102047/Non-residents-income-based-Wage-Tax-refund-form-2022.pdf, https://www.phila.gov/media/20240131144329/2023-Wage-Tax-refund-salaried-employees-form-and-instructions.pdf, https://www.phila.gov/media/20231024112123/UPDATED-Wage-Tax-policy-guidance-for-non-resident-employees.pdf)"
What is the normal processing time for a refund request reviewed by the Department of Revenue?,I cannot answer questions that are not about Wages and taxes other than PA.,List()
What were the tax rates for residents of Philadelphia in the year 2018?,"I'm sorry for any confusion, but the context provided does not include the tax rates for residents of Philadelphia in the year 2018. The context includes tax rates for the year 2023 and instructions for filing a refund for non-residents for the tax year 2022. If you have any questions about the 2018 tax rates, I can try to find that information for you, but I would need to look it up in a different source.","List(https://www.phila.gov/media/20240131144329/2023-Wage-Tax-refund-salaried-employees-form-and-instructions.pdf, https://www.phila.gov/media/20201105092132/Wage-Tax-non-resident-COVID-19-guidance-110520.pdf, https://www.phila.gov/media/20230206102047/Non-residents-income-based-Wage-Tax-refund-form-2022.pdf, https://www.phila.gov/media/20231024112123/UPDATED-Wage-Tax-policy-guidance-for-non-resident-employees.pdf)"
What's the due date for 12/01/21-12/15/21 semi-monthly wage tax vs. Dec 2021 monthly wage tax as per 2021 schedule?,"The due date for the semi-monthly wage tax period of 12/01/21-12/15/21 is 12/20/21, while the due date for the monthly wage tax for December 2021 is also 12/20/21.","List(https://www.phila.gov/media/20201021083908/2021-Wage-semi-monthly-tax-due-dates.pdf, https://www.phila.gov/media/20201021083906/Wage-Tax-due-dates-2021-schedule-v1.pdf, https://www.phila.gov/media/20201021084110/2021-Wage-Monthly-and-Quarterly-Tax-Due-dates.pdf, https://www.phila.gov/media/20191030085637/Wage-Tax-due-dates-2020-schedule-v1.pdf)"
When should non-8hr workday hrs be logged in the Non-res. Wage Tax Refund Petition & Employer Cert. Template?,"Non-work days or hours, such as weekends, vacation, holidays, sick or any type of leave time, should be included in the calculation of the wage tax refund for non-residents of Philadelphia. However, the provided context does not contain a specific template for the Non-res. Wage Tax Refund Petition & Employer Cert. Template. In general, non-work hours can be included in the calculation of the wage tax refund by determining the number of hours worked in Philadelphia and subtracting the non-work hours. The resulting number is the amount of time spent working in Philadelphia, which can be used to calculate the wage tax refund. It is recommended to consult the instructions provided with the Non-res. Wage Tax Refund Petition & Employer Cert. Template for specific guidance on how to log non-work hours. If the template requires the entry of hours worked, non-work hours can be included as hours not worked in Philadelphia. If the template requires the entry of wages earned in Philadelphia, non-work hours can be subtracted from the total wages earned to determine the amount of wages earned in Philadelphia.","List(https://www.phila.gov/media/20191218101514/Wage-Tax-refund-petition-salaried-employ